In [ ]:
# Problem statement
# 14 controls
# 1 target
# 5 auxilliary 

# b0-13, t, a0-4

# Design 1: (3-4-3)
# 4 3-bit in place control gates (b0-11)->(a0-3)
# 1 4-bit in place control gate taking 2 remaining bits and 2 auxiliary bits (b12-13, a0-1) -> a4
# 3 3-bit in place control gate that takes (a2-4) -> t


In [1]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram 

In [33]:
control_qubits = QuantumRegister(14, name="c")
target_qubit = QuantumRegister(1, name='t')
ancilla_qubits = QuantumRegister(5, name='a')
classical_bits = ClassicalRegister(1, name='out')
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)

#qc.mcx()

In [34]:
for i in range(14):
    qc.x(control_qubits[i])

for i in range(4):
    qc.mcx(control_qubits[3*i:3*(i+1)], ancilla_qubits[i])

qc.mcx(control_qubits[12:14] + ancilla_qubits[0:2], ancilla_qubits[4])
qc.mcx(ancilla_qubits[2:5], target_qubit[0])
qc.measure(target_qubit[0], classical_bits[0])


In [35]:
qc.draw()

┌───┐                                 
  c_0: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_1: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_2: ┤ X ├──■──────────────────────────────
       ├───┤  │                              
  c_3: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_4: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_5: ┤ X ├──┼────■─────────────────────────
       ├───┤  │    │                         
  c_6: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_7: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_8: ┤ X ├──┼────┼────■────────────────────
       ├───┤  │    │    │                    
  c_9: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_10: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_11: ┤ X ├──┼────┼────┼────■───────────────
       ├───┤  │    │    │    │               
 c_12: ┤ X ├──┼────┼────┼────┼────■──────────
       ├───┤  │    │    │    │    │          
 c_13: ┤ X ├──┼────┼────┼────┼────■──────────
       └───┘  │    │    │    │    │  ┌───┐┌─┐
    t: ───────┼────┼────┼────┼────┼──┤ X ├┤M├
            ┌─┴─┐  │    │    │    │  └─┬─┘└╥┘
  a_0: ─────┤ X ├──┼────┼────┼────■────┼───╫─
            └───┘┌─┴─┐  │    │    │    │   ║ 
  a_1: ──────────┤ X ├──┼────┼────■────┼───╫─
                 └───┘┌─┴─┐  │    │    │   ║ 
  a_2: ───────────────┤ X ├──┼────┼────■───╫─
                      └───┘┌─┴─┐  │    │   ║ 
  a_3: ────────────────────┤ X ├──┼────■───╫─
                           └───┘┌─┴─┐  │   ║ 
  a_4: ─────────────────────────┤ X ├──■───╫─
                                └───┘      ║ 
out: 1/════════════════════════════════════╩═
                                           0

In [16]:
qc = transpile(qc, basis_gates=['u', 'cx'])

In [ ]:
qc.draw()

In [36]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'])
qobj = assemble(qc_transpiled)
result = aer_simulator.run(qobj).result()
print(result.get_counts())

{'1': 1024}


In [25]:
qc_transpiled.draw()

global phase: 0
        ┌────────────┐                                                   »
 c_0: ──┤ U(0,0,π/8) ├───■───────────────────■────────────────────────■──»
        ├────────────┤ ┌─┴─┐┌─────────────┐┌─┴─┐                      │  »
 c_1: ──┤ U(0,0,π/8) ├─┤ X ├┤ U(0,0,-π/8) ├┤ X ├──■───────────────────┼──»
        ├────────────┤ └───┘└─────────────┘└───┘┌─┴─┐┌─────────────┐┌─┴─┐»
 c_2: ──┤ U(0,0,π/8) ├──────────────────────────┤ X ├┤ U(0,0,-π/8) ├┤ X ├»
        ├────────────┤                          └───┘└─────────────┘└───┘»
 c_3: ──┤ U(0,0,π/8) ├───■───────────────────■────────────────────────■──»
        ├────────────┤ ┌─┴─┐┌─────────────┐┌─┴─┐                      │  »
 c_4: ──┤ U(0,0,π/8) ├─┤ X ├┤ U(0,0,-π/8) ├┤ X ├──■───────────────────┼──»
        ├────────────┤ └───┘└─────────────┘└───┘┌─┴─┐┌─────────────┐┌─┴─┐»
 c_5: ──┤ U(0,0,π/8) ├──────────────────────────┤ X ├┤ U(0,0,-π/8) ├┤ X ├»
        ├────────────┤                          └───┘└─────────────┘└───┘»
 c_6: ──┤ U(0,0,π/8) ├───■───────────────────■────────────────────────■──»
        ├────────────┤ ┌─┴─┐┌─────────────┐┌─┴─┐                      │  »
 c_7: ──┤ U(0,0,π/8) ├─┤ X ├┤ U(0,0,-π/8) ├┤ X ├──■───────────────────┼──»
        ├────────────┤ └───┘└─────────────┘└───┘┌─┴─┐┌─────────────┐┌─┴─┐»
 c_8: ──┤ U(0,0,π/8) ├──────────────────────────┤ X ├┤ U(0,0,-π/8) ├┤ X ├»
        ├────────────┤                          └───┘└─────────────┘└───┘»
 c_9: ──┤ U(0,0,π/8) ├───■───────────────────■────────────────────────■──»
        ├────────────┤ ┌─┴─┐┌─────────────┐┌─┴─┐                      │  »
c_10: ──┤ U(0,0,π/8) ├─┤ X ├┤ U(0,0,-π/8) ├┤ X ├──■───────────────────┼──»
        ├────────────┤ └───┘└─────────────┘└───┘┌─┴─┐┌─────────────┐┌─┴─┐»
c_11: ──┤ U(0,0,π/8) ├──────────────────────────┤ X ├┤ U(0,0,-π/8) ├┤ X ├»
        └────────────┘                          └───┘└─────────────┘└───┘»
c_12: ───────────────────────────────────────────────────────────────────»
                                                                         »
c_13: ───────────────────────────────────────────────────────────────────»
      ┌───────────────┐                                                  »
   t: ┤ U(π/2,π/8,-π) ├──────────────────────────────────────────────────»
      ├───────────────┤                                                  »
 a_0: ┤ U(π/2,π/8,-π) ├──────────────────────────────────────────────────»
      ├───────────────┤                                                  »
 a_1: ┤ U(π/2,π/8,-π) ├──────────────────────────────────────────────────»
      ├───────────────┤                                                  »
 a_2: ┤ U(π/2,π/8,-π) ├──────────────────────────────────────────────────»
      ├───────────────┤                                                  »
 a_3: ┤ U(π/2,π/8,-π) ├──────────────────────────────────────────────────»
      └─┬────────────┬┘                                                  »
 a_4: ──┤ U(π/2,0,π) ├───────────────────────────────────────────────────»
        └────────────┘                                                   »
«                                                                 »
« c_0: ────────────────────────────────────■──────────────────────»
«                                          │                      »
« c_1: ────────────────■───────────────────┼──────────────────────»
«      ┌────────────┐┌─┴─┐┌─────────────┐┌─┴─┐                    »
« c_2: ┤ U(0,0,π/8) ├┤ X ├┤ U(0,0,-π/8) ├┤ X ├──■─────────────────»
«      └────────────┘└───┘└─────────────┘└───┘  │                 »
« c_3: ────────────────────────────────────■────┼─────────────────»
«                                          │    │                 »
« c_4: ────────────────■───────────────────┼────┼─────────────────»
«      ┌────────────┐┌─┴─┐┌─────────────┐┌─┴─┐  │                 »
« c_5: ┤ U(0,0,π/8) ├┤ X ├┤ U(0,0,-π/8) ├┤ X ├──┼────■────────────»
«      └────────────┘└───┘└─────────────┘└───┘  │    │            »
« c_6: ──────────────────